# EEG Batch Import

This notebook batch imports all the pairs of `xdf` and `json` files in the Data folder. Note that one must first create the `json` files by running the [eeg_processing](eeg_preprocessing.ipynb) notebook first.

## Import libraries

In [ ]:
# Standard libraries
import mne
import json
import numpy as np
from mne.preprocessing import ICA

# Custom libraries
from Functions import import_data
from Functions import data_tools

## Select files to import

Add the name of the files to import. Note that each file in the list must have a `xdf` and `json` file with that name.

In [ ]:
# Settings
# - List of files to import and pre-process
files_to_import = [
    "sub-P003_ses-S001_task-T1_run-001_eeg"
]

save_npy = True    # Boolean to save numpy epochs 

## Import and preprocess data



In [ ]:
# Preallocate variables 
eeg_mne = [None] * len(files_to_import) # List of MNE objects with raw data

for file in files_to_import:
    # Import settings saved in json
    with open(f"Data\{file}.json", "r") as file_object:
        settings = json.load(file_object)

    # Import EEG data
    [eeg_ts, eeg_data, eeg_fs] = import_data.read_xdf(
        f"Data\{file}.xdf",
        picks=settings["ch_names"]
        )
    
    # Create MNE array
    info = mne.create_info(settings["ch_names"], eeg_fs, ch_types='eeg')    # Create info properties
    mne_raw = mne.io.RawArray(eeg_data, info=info)                          # Create MNE raw array
    mne_raw.set_montage('standard_1020')                                    # Set montage to array

    # Apply band-pass filter
    filt_raw = mne_raw.copy().filter(
        l_freq = settings["lpf_fc"],
        h_freq = settings["hpf_fc"],
        picks = settings["ch_names"]
        )
    
    # Create ICA
    ica = ICA(
        n_components = settings["n_components"],
        max_iter = settings["max_iter"],
        random_state = settings["random_state"]
        )
    ica.fit(filt_raw, picks=settings["ch_names"])

    # Remove ICA components
    ica.exclude = settings["ica_exclude"]
    ica.apply(filt_raw)

    # Epoch data
    [marker_ts, markers] = import_data.read_xdf_unity_markers(f"Data\{file}.xdf")
    [eeg_epochs, epoch_labels] = data_tools.epochs_from_unity_markers(
        eeg_time = eeg_ts,
        eeg_data = filt_raw.get_data(),
        marker_time = marker_ts,
        marker_data = markers
        )
    
    # - Get stimuli and frequencies labels
    fixed_labels = data_tools.fix_labels(epoch_labels)          # Fix labels that might be incorrect in pilot trials
    dict_of_stimuli = data_tools.get_tvep_stimuli(fixed_labels) # Dictionary of unique stimulus
    dict_of_freqs = {0:"2", 1:"9.6", 2:"16", 3:"36"}            # Dictionary of stimulus frequencies

    # - Create array of eeg epochs organized as [stimuli, freq, chans, samples]
    eeg_epochs_organized = data_tools.epochs_stim_freq(
        eeg_epochs = eeg_epochs,
        labels = fixed_labels,
        stimuli = dict_of_stimuli,
        freqs = dict_of_freqs
        )
    
    # Save data to NPY file
    if (save_npy):
        np.save(
            f"Data\{file}-batch.npy",
            eeg_epochs_organized,
            )   